In [2]:
import warnings
import pandas as pd
import numpy as np

from jre_utils.datapath import factor_data_paths, DATA_DIRECTORY_PATH

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
migration_unprocessed_path = factor_data_paths["unprocessed"]["migration"]["municipality"]
additional_migration_unprocessed_path = f"{DATA_DIRECTORY_PATH}/municipality/unprocessed/migration/migration_2020_2022.csv"
new_migration_unprocessed_path = f"{DATA_DIRECTORY_PATH}/municipality/unprocessed/migration/migration_1996_2022.csv"

migration_df = pd.read_csv(migration_unprocessed_path)
additional_migration_df = pd.read_csv(additional_migration_unprocessed_path)

In [7]:
area_code_to_area_df = migration_df[["area_code", "area"]].drop_duplicates()
area_code_to_area_df.set_index("area_code")["area"].to_dict()


{1100: 'Hokkaido Sapporo-shi',
 1101: 'Hokkaido Sapporo-shi Chuo-ku',
 1102: 'Hokkaido Sapporo-shi Kita-ku',
 1103: 'Hokkaido Sapporo-shi Higashi-ku',
 1104: 'Hokkaido Sapporo-shi Shiroishi-ku',
 1105: 'Hokkaido Sapporo-shi Toyohira-ku',
 1106: 'Hokkaido Sapporo-shi Minami-ku',
 1107: 'Hokkaido Sapporo-shi Nishi-ku',
 1108: 'Hokkaido Sapporo-shi Atsubetsu-ku',
 1109: 'Hokkaido Sapporo-shi Teine-ku',
 1110: 'Hokkaido Sapporo-shi Kiyota-ku',
 1202: 'Hokkaido Hakodate-shi',
 1203: 'Hokkaido Otaru-shi',
 1204: 'Hokkaido Asahikawa-shi',
 1205: 'Hokkaido Muroran-shi',
 1206: 'Hokkaido Kushiro-shi',
 1207: 'Hokkaido Obihiro-shi',
 1208: 'Hokkaido Kitami-shi',
 1209: 'Hokkaido Yubari-shi',
 1210: 'Hokkaido Iwamizawa-shi',
 1211: 'Hokkaido Abashiri-shi',
 1212: 'Hokkaido Rumoi-shi',
 1213: 'Hokkaido Tomakomai-shi',
 1214: 'Hokkaido Wakkanai-shi',
 1215: 'Hokkaido Bibai-shi',
 1216: 'Hokkaido Ashibetsu-shi',
 1217: 'Hokkaido Ebetsu-shi',
 1218: 'Hokkaido Akabira-shi',
 1219: 'Hokkaido Mombetsu-s

In [8]:
import json

area_code_to_area_path = f"{DATA_DIRECTORY_PATH}/core_scraped/area_code_to_area.json"
with open(area_code_to_area_path, 'w') as fp:
    json.dump(area_code_to_area_df.set_index("area_code")["area"].to_dict(), fp)

In [65]:
additional_migration_df = additional_migration_df[additional_migration_df["year"] == 2022]
additional_migration_df

,year,area_code,area,in_migrations,out_migrations,net_migrations
0,2022,0,Japan,4775207,4775207,0
1,2022,1,All shi,4452732,4427106,25626
2,2022,2,All gun,322475,348101,-25626
3,2022,1000,Hokkaido,224525,228548,-4023
4,2022,1001,Hokkaido shi,193184,191190,1994
...,...,...,...,...,...,...
2381,2022,47370,Miyako-gun,58,60,-2
2382,2022,47375,Tarama-son,58,60,-2
2383,2022,47380,Yaeyama-gun,798,783,15
2384,2022,47381,Taketomi-cho,573,587,-14


In [66]:
columns = ["year", "area_code", "area", "in_migrations", "out_migrations"]
additional_migration_df = additional_migration_df.merge(area_code_to_area_df, on=["area_code"], suffixes=["_x", ""])[columns]
additional_migration_df

,year,area_code,area,in_migrations,out_migrations
0,2022,1100,Hokkaido Sapporo-shi,114194,105217
1,2022,1101,Hokkaido Sapporo-shi Chuo-ku,21118,17618
2,2022,1102,Hokkaido Sapporo-shi Kita-ku,14532,13683
3,2022,1103,Hokkaido Sapporo-shi Higashi-ku,14109,13484
4,2022,1104,Hokkaido Sapporo-shi Shiroishi-ku,13322,12717
...,...,...,...,...,...
1911,2022,47361,Okinawa-ken Kumejima-cho,390,506
1912,2022,47362,Okinawa-ken Yaese-cho,1650,1180
1913,2022,47375,Okinawa-ken Tarama-son,58,60
1914,2022,47381,Okinawa-ken Taketomi-cho,573,587


In [72]:
updated_migration_df = pd.concat([additional_migration_df, migration_df])
updated_migration_df = updated_migration_df.sort_values(by=["year", "area_code"], ascending=[False, True])
updated_migration_df

,year,area_code,area,in_migrations,out_migrations
0,2022,1100,Hokkaido Sapporo-shi,114194,105217
1,2022,1101,Hokkaido Sapporo-shi Chuo-ku,21118,17618
2,2022,1102,Hokkaido Sapporo-shi Kita-ku,14532,13683
3,2022,1103,Hokkaido Sapporo-shi Higashi-ku,14109,13484
4,2022,1104,Hokkaido Sapporo-shi Shiroishi-ku,13322,12717
...,...,...,...,...,...
49810,1996,47361,Okinawa-ken Kumejima-cho,549,619
49811,1996,47362,Okinawa-ken Yaese-cho,1539,1212
49812,1996,47375,Okinawa-ken Tarama-son,52,98
49813,1996,47381,Okinawa-ken Taketomi-cho,430,441


In [73]:
updated_migration_df.to_csv(new_migration_unprocessed_path, index=False)
